# Лабораторная работа 2.

# Задача 1

Загружаем корпус текстов из файла train.json. Заглянем в набор данных с помощью data.head(10), чтобы представлять, как выглядит данный корпус текстов. Разобьем исходный набор данных на тренировочную и тестовую выборку в соотношении 70/30. В X будут содержаться наши тексты, представленные как вектор из частотностей слов. А в У - оценки тональности, но каждой из них мы назначим числовое значение.

In [23]:
import numpy as np
import os
import pandas as pd

#загружаем дата-сет
data = pd.read_json('train.json', encoding = 'utf-8')
data.head(10)

from sklearn.model_selection import train_test_split

#присваиваем классам числовое значение
y = {'negative': -1, 'neutral': 0, 'positive': 1} 
data['y'] = data['sentiment'].map(lambda x: y[x])

#делим данные на x и y, в x будут тексты, а в у - класс
data_Y = data['y'].values
data_X = data['text'].values

#делим данные на тренировочную и тестовую выборку, тестовая выборка - 30% (здесь взята только часть примеров, а не весь корпус - это связано с мощностью компьютера)
dataY_train, dataY_test, dataX_train, dataX_test = train_test_split(data_Y, data_X, test_size=0.3, shuffle=True)
dataY_train, dataY_test, dataX_train, dataX_test = dataY_train[:700], dataY_test[:300], dataX_train[:700], dataX_test[:300]

# Задача 2

Создадим “baseline” (отправную точку) - достаточно простую модель, используя сравнительно несложную схему репрезентации текста. Будем представлять наши тексты как вектор, включающий в себя частотности уникальных слов.

In [24]:
from sklearn.feature_extraction.text import CountVectorizer 

#для нашей тренировочной выборки создаем словарь частотностей всех уникальных словоформ 
vectorizer = CountVectorizer(min_df = 0, lowercase = True) 
vectorizer.fit(dataX_train)

#преобразуем полученные данные в вектор
X_train = vectorizer.transform(dataX_train).toarray()

test_vectorizer = CountVectorizer(min_df = 0, lowercase = True, vocabulary=vectorizer.vocabulary_) 
test_vectorizer.fit(dataX_test)

#представляем в виде вектора каждый текст из тестовой выборки, элементу с каким-либо индексом соответствует частотность слова, которое под этим же индексом находится в нашем словаре, в данном тексте
X_test = test_vectorizer.transform(dataX_test).toarray()

print(X_train[:10])
print(X_test[:10])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Задачи 3-4

Нужно использовать как минимум три классификатора. Как минимум два из них будут иметь возможность возвращать важность атрибутов. Как минимум один будет иметь возможность получать вероятности классов. Нужно решить задачу классификации, рассмотрев сначала как задачу многоклассовой классификации (позитив/негатив/нейтральный класс). 

Сначала попробуем решить задачу многоклассовой классификации. Применим RandomDorest, ExtraTrees и линейный SVM.

In [25]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

linsvc_clf = LinearSVC(C=5, loss="hinge", penalty = 'l2', multi_class = 'ovr')

#применим gridsearch для поиска оптимальных значений гиперпараметров
param_grid = {'C': list(range(1, 5)), 'loss': ['hinge', 'squared_hinge']}
grid_search_cv = GridSearchCV(LinearSVC(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred1 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred1)

print(accuracy_score(dataY_test, y_pred1))

0.5933333333333334


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

rnd_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
rnd_clf.fit(X_train, dataY_train)

#создадим конвейер для векторайзера и классификатора
our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('RandomForestClassifier', RandomForestClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(RandomForestClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred2 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred2)

#выводим метрики качества, accuracy, важность каждого слова, вероятности классов и список слов, на которые больше всего опираетсямодель при выборе класса
print(accuracy_score(dataY_test, y_pred2))
print(rnd_clf.feature_importances_)
print(rnd_clf.predict_proba(X_test))
np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['RandomForestClassifier'].feature_importances_ != 0)[0]]

0.4666666666666667
[0. 0. 0. ... 0. 0. 0.]
[[0.17298522 0.50196701 0.32504777]
 [0.17233262 0.50008278 0.3275846 ]
 [0.17621644 0.51208972 0.31169384]
 [0.17429831 0.50801258 0.31768911]
 [0.17156481 0.50379776 0.32463742]
 [0.16432562 0.50282169 0.33285269]
 [0.17226095 0.51484634 0.31289272]
 [0.17243226 0.49847757 0.32909017]
 [0.1864296  0.50104514 0.31252526]
 [0.17628262 0.50133168 0.3223857 ]
 [0.17291714 0.50819463 0.31888823]
 [0.17879156 0.51788878 0.30331966]
 [0.18425416 0.50657659 0.30916926]
 [0.17454964 0.5118657  0.31358466]
 [0.18936213 0.50708793 0.30354994]
 [0.1707189  0.50830341 0.3209777 ]
 [0.17341369 0.50898558 0.31760073]
 [0.17731089 0.50203321 0.3206559 ]
 [0.16592766 0.49905335 0.33501899]
 [0.17456252 0.51473349 0.31070399]
 [0.16326924 0.48635796 0.3503728 ]
 [0.17086808 0.51105605 0.31807586]
 [0.17568118 0.50256915 0.32174968]
 [0.17360885 0.51089764 0.31549351]
 [0.17828921 0.5246732  0.29703759]
 [0.17780191 0.50407222 0.31812588]
 [0.16496468 0.506109

array(['00', '04', '05', ..., 'қтж', 'құрмет', 'үшін'], dtype='<U66')

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

ext_clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
ext_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('ExtraTreesClassifier', ExtraTreesClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(ExtraTreesClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred3 = grid_search_cv.predict(X_test)

print(accuracy_score(dataY_test, y_pred3))
print(ext_clf.feature_importances_)
print(ext_clf.predict_proba(X_test))
np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['ExtraTreesClassifier'].feature_importances_ != 0)[0]]

0.47333333333333333
[0. 0. 0. ... 0. 0. 0.]
[[0.17289671 0.50849552 0.31860777]
 [0.16941997 0.50112626 0.32945377]
 [0.1741058  0.50896006 0.31693415]
 [0.17391328 0.50860444 0.31748228]
 [0.17497727 0.50942147 0.31560126]
 [0.16736544 0.49921916 0.33341539]
 [0.17227958 0.51344132 0.3142791 ]
 [0.18790904 0.48459826 0.32749269]
 [0.17668274 0.50861052 0.31470674]
 [0.1741058  0.50896006 0.31693415]
 [0.1741058  0.50896006 0.31693415]
 [0.1741058  0.50896006 0.31693415]
 [0.1824514  0.50648947 0.31105913]
 [0.1741058  0.50896006 0.31693415]
 [0.18590026 0.50631844 0.3077813 ]
 [0.16874923 0.49897915 0.33227162]
 [0.17426476 0.50161226 0.32412298]
 [0.1741058  0.50896006 0.31693415]
 [0.17228227 0.50693404 0.32078369]
 [0.18061653 0.5021319  0.31725158]
 [0.16414041 0.49459419 0.3412654 ]
 [0.17282769 0.51043099 0.31674132]
 [0.182613   0.4989562  0.3184308 ]
 [0.1741058  0.50896006 0.31693415]
 [0.17322197 0.50575603 0.321022  ]
 [0.17149126 0.503495   0.32501374]
 [0.17227419 0.50784

array(['01', '026', '04', ..., 'қазақстан', 'үшін', 'ұлы'], dtype='<U66')

Пока лучший результат показывает линейный SVM (accuracy = 0.59).

Затем попробуем решить задачу бинарной классификации. Для этого сначала нужно присвоить другие значения оценкам тональности, чтобы их было всего два: 1 и -1. Здесь применим классификаторы RandomForest, ExtraTrees и логистическую регрессию.

In [6]:
#зададим новые численные значения для классов
y = {'negative': -1, 'neutral': 1, 'positive': 1} 
data['y'] = data['sentiment'].map(lambda x: y[x])

data_Y = data['y'].values
data_X = data['text'].values

dataY_train, dataY_test, dataX_train, dataX_test = train_test_split(data_Y, data_X, test_size=0.3)
dataY_train, dataY_test, dataX_train, dataX_test = dataY_train[:700], dataY_test[:300], dataX_train[:700], dataX_test[:300]

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

#выполняем те же самые действия, но для двух классов
rnd_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
rnd_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('RandomForestClassifier', RandomForestClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(RandomForestClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred2 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred2)

print(accuracy_score(dataY_test, y_pred2))
print(rnd_clf.feature_importances_)
print(rnd_clf.predict_proba(X_test))
print(classification_report(y_true=dataY_test, y_pred=y_pred2))

np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['RandomForestClassifier'].feature_importances_ != 0)[0]]

0.8666666666666667
[0. 0. 0. ... 0. 0. 0.]
[[0.16982263 0.83017737]
 [0.17840654 0.82159346]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.1740095  0.8259905 ]
 [0.176361   0.823639  ]
 [0.16310269 0.83689731]
 [0.16159364 0.83840636]
 [0.16310269 0.83689731]
 [0.17113738 0.82886262]
 [0.16310269 0.83689731]
 [0.16849734 0.83150266]
 [0.17155645 0.82844355]
 [0.16310269 0.83689731]
 [0.19066833 0.80933167]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16533161 0.83466839]
 [0.16159364 0.83840636]
 [0.16310269 0.83689731]
 [0.17161425 0.82838575]
 [0.16646324 0.83353676]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.17693027 0.82306973]
 [0.17815876 0.82184124]
 [0.16722502 0.83277498]
 [0.17132267 0.82867733]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.17155645 0.82844355]
 [0.17168651 0.82831349]
 [0.163

array(['00', '047', '10', '103', '11', '1126', '115', '12', '128', '13',
       '1306', '14', '15', '16', '18', '19', '192', '20', '2007', '2009',
       '2013', '2015', '2016', '2017', '208', '21', '23', '234', '25',
       '250', '259', '27', '29', '300er', '318', '333', '36', '404',
       '4187', '44', '45', '49', '500', '53', '54', '644', '659', '662',
       '69', '727', '7373', '738', '758', '803', '84', '900', '95', '990',
       'bank', 'banker', 'bloomberg', 'emitters', 'google', 'https',
       'hyundai', 'inform', 'kalina', 'kapital', 'kapitala', 'kase',
       'kaz', 'kazakhstan', 'kegoc', 'kz', 'kzbbb', 'lot', 'ministr',
       'pavon', 'pdf', 'quot', 'raquo', 'renault', 'rezonans', 'shell',
       'zonakz', 'абаиловского', 'аварийная', 'августа', 'автобизнеса',
       'автобусы', 'автомобилей', 'автомобильной', 'авторизацию',
       'агентов', 'агромашхолдинг', 'административной',
       'администрирования', 'айтимбетов', 'акимата', 'акишева',
       'актауский', 'актюби

In [26]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report

ext_clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
ext_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('ExtraTreesClassifier', ExtraTreesClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(ExtraTreesClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred3 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred3)

print(accuracy_score(dataY_test, y_pred3))
print(ext_clf.feature_importances_)
print(ext_clf.predict_proba(X_test))
print(classification_report(y_true=dataY_test, y_pred=y_pred3))

np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['ExtraTreesClassifier'].feature_importances_ != 0)[0]]

0.8766666666666667
[0. 0. 0. ... 0. 0. 0.]
[[0.16711212 0.83288788]
 [0.1686411  0.8313589 ]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.1686411  0.8313589 ]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.18539103 0.81460897]
 [0.17543108 0.82456892]
 [0.16535878 0.83464122]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.1736363  0.8263637 ]
 [0.1686411  0.8313589 ]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.1686901  0.8313099 ]
 [0.16711212 0.83288788]
 [0.17087393 0.82912607]
 [0.19209775 0.80790225]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.17535878 0.82464122]
 [0.19868684 0.80131316]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.17210972 0.82789028]
 [0.168

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array(['000', '01', '03', ..., 'январе', 'января', 'қазына'], dtype='<U32')

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('LogisticRegression', LogisticRegression(penalty='l1'))]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'C': list(range(1, 5)), 'penalty': ['l1', 'l2']}
grid_search_cv = GridSearchCV(LogisticRegression(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred4 = grid_search_cv.predict(X_test)
print(accuracy_score(dataY_test, y_pred4))
print(classification_report(y_true=dataY_test, y_pred=y_pred4))

np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['LogisticRegression'].coef_ != 0)[1]]

0.81
             precision    recall  f1-score   support

         -1       0.27      0.06      0.10        52
          1       0.83      0.97      0.89       248

avg / total       0.73      0.81      0.76       300



array(['10', '100', '11', '13', '2012', '2015', '2016', '2016г', '22',
       '25', '28', '30', '52', '60', 'auto', 'bombardier', 'cs300',
       'enpf', 'invest', 'kase', 'kegoc', 'kookmin', 'lada', 'mega',
       'rusal', 'zakon', 'авиакомпании', 'авиационного', 'азия', 'акишев',
       'акций', 'алматинской', 'алматы', 'ао', 'астана', 'байтерек',
       'банк', 'безопасности', 'бизнес', 'бизнеса', 'бишимбаев', 'блох',
       'будут', 'был', 'были', 'вв', 'ведомстве', 'вице', 'вич',
       'вконтакте', 'вокзале', 'все', 'встречи', 'где', 'глава', 'года',
       'году', 'города', 'гривень', 'даже', 'данный', 'двд', 'двое',
       'дела', 'дело', 'денег', 'деньги', 'директоров', 'для', 'до',
       'долг', 'должен', 'дома', 'евразийский', 'его', 'ед', 'енпф',
       'еще', 'же', 'завод', 'заместитель', 'иванов', 'из', 'изменения',
       'им', 'именно', 'имеют', 'интер', 'информацию', 'итогам', 'их',
       'казахстан', 'казахстана', 'казахстане', 'казахстанской',
       'казинвестбанк

В случае с бинарной классификацией лучшим по точности оказался классификатор Random Forest. (accuracy = 0.826667)

Можно заметить, что хоть и большинство слов, на которые опирается модель, определяя тональность, схожи, есть также и различные слова для каждого случая. Также среди слов, на которые модель обращает внимание, встречается очень много лишних слов, которые, по сути, не должны влиять на определение тональности текста, например, часто встречающиеся слова в целом, числительные или предлоги.

# Задача 5

Попробуем изменить схему препроцессинга. Проведем лемматизацию атрибутов.


In [10]:
import numpy as np
import os
import pandas as pd

data = pd.read_json('train.json', encoding = 'utf-8')
data.head(10)

from sklearn.model_selection import train_test_split

y = {'negative': -1, 'neutral': 0, 'positive': 1} 
data['y'] = data['sentiment'].map(lambda x: y[x])

data_Y = data['y'].values
data_X = data['text'].values

dataY_train, dataY_test, dataX_train, dataX_test = train_test_split(data_Y, data_X, test_size=0.3, shuffle=True)
dataY_train, dataY_test, dataX_train, dataX_test = dataY_train[:700], dataY_test[:300], dataX_train[:700], dataX_test[:300]

#преобразуем наши тексты в списки слов
X_train = []
for x in dataX_train:
    x = x.split()
    X_train.append(x)
    
X_test = []
for x in dataX_test:
    x = x.split()
    X_test.append(x)

In [11]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
X_train_l = []
X_test_l = []

#пройдем по каждому слову и вместо него возьмем его начальную форму
for text in X_train:
    lemmas = []
    for word in text:
        p = morph.parse(word)[0]
        lemmas.append(p.normal_form)
    X_train_l.append(lemmas)
for text in X_test:
    lemmas = []
    for word in text:
        p = morph.parse(word)[0]
        lemmas.append(p.normal_form)
    X_test_l.append(lemmas)

In [12]:
print(X_train_l[:1])

[['монополист', 'объявить', 'о', '50%', 'скидка', 'на', 'путешествие', '31', 'декабря,', 'сообщать', 'портал', 'мой', 'город.', '«ао', '«пассажирский', 'перевозки»', 'в', 'преддверие', 'новое', 'год', 'объявить', 'скидка', 'на', 'скоростной', 'поезд', '«тулпар-тальго».', 'как', 'сказать', 'на', 'они', 'сайте,', 'на', 'весь', 'поезд', '«тулпар-тальго»', '(кром', 'поезд', '№', '3/4', 'астан', '—', 'алматы,', '№', '25/26', 'алматы', '—', 'шымкент)', 'с', 'отправление', '31', 'декабрь', 'билет', 'можно', 'быть', 'приобрести', 'в', 'два', 'раз', 'дешевле.', 'например,', 'билет', 'на', 'поезд', '№', '701/702,', '№', '706/705', 'сообщение', 'астан', '—', 'алматы', 'на', '31', 'декабрь', 'быть', 'стоить', 'от', '6', '000', 'тенге,', 'шымкент', '—', 'астан', '—', 'от', '5', '500', 'тенге,', 'кызылорд', '—', 'астан', '—', 'от', '8', '500', 'тенге,', 'астан', '—', 'защита', '—', 'от', '5', '300', 'тенге,', 'алматы', '—', 'защита', '—', 'от', '4', '900', 'тенге,', 'алматы', '—', 'петропавловск', '

А также очистим тексты от стоп-слов, используя nltk. Кроме стоп-слов из nltk удалим еще числительные, поскольку они тоже иногда препятствуют правильному определению тональности.

In [13]:
import nltk
from nltk.corpus import stopwords

for text in X_train_l:
    for word in text:
        if word in stopwords.words('russian') or word.isdigit() == True: 
            text.remove(word)
    
for text in X_test_l:
    for word in text:
        if word in stopwords.words('russian') or word.isdigit() == True:
            text.remove(word)

In [14]:
print(X_train_l[:1])

[['монополист', 'объявить', '50%', 'скидка', 'путешествие', 'декабря,', 'сообщать', 'портал', 'город.', '«ао', '«пассажирский', 'перевозки»', 'преддверие', 'новое', 'год', 'объявить', 'скидка', 'скоростной', 'поезд', '«тулпар-тальго».', 'сказать', 'они', 'сайте,', 'весь', 'поезд', '«тулпар-тальго»', '(кром', 'поезд', '№', '3/4', 'астан', '—', 'алматы,', '№', '25/26', 'алматы', '—', 'шымкент)', 'отправление', 'декабрь', 'билет', 'приобрести', 'два', 'дешевле.', 'например,', 'билет', 'поезд', '№', '701/702,', '№', '706/705', 'сообщение', 'астан', '—', 'алматы', '31', 'декабрь', 'стоить', '6', 'тенге,', 'шымкент', '—', 'астан', '—', '5', 'тенге,', 'кызылорд', '—', 'астан', '—', '8', 'тенге,', 'астан', '—', 'защита', '—', '5', 'тенге,', 'алматы', '—', 'защита', '—', '4', 'тенге,', 'алматы', '—', 'петропавловск', '—', '7', 'тенге.', 'однако', 'железнодорожник', 'предупредили,', 'близкий', 'дата', 'отправление', 'поезд', 'при', 'увеличение', 'спрос', 'стоимость', 'билет', 'быть', 'расти.']]


In [15]:
#преобразуем списки слов снова в тексты, чтобы вернуться к исходной форме
X_train_l_new = []
X_test_l_new = []

for text in X_train_l:
    text_str = " ".join(text)
    X_train_l_new.append(text_str)
    
for text in X_test_l:
    text_str = " ".join(text)
    X_test_l_new.append(text_str)

In [16]:
print(X_test_l_new[:1])

['астана. казинформ - президент казахстан нурсултан назарбаев выразить недовольство работа создание условие инвесторов. это заявить сегодня ход расширить заседание правительства, передавать корреспондент миа «казинформ». глава государство отметил, министерство инвестиция развитие являться ответственный вопрос привлечение инвестор продвижение экспорта. «ты же, я провозить встреча «капитан бизнеса» весь страна был. недавно саудовский аравия были, эмиратах, это японии, южный корее, америке. я это делаю. весь проявлять большой заинтересованность казахстану. когда приходить сюда (потенциальный инвестор - прим. автора), встречаться. никто решать вопросы. приезжать говорят, наш ребёнок учится, лечится, делать? должный показать им, у казахстан создать весь условие инвесторов», - сказать нурсултан назарбаев, обращаться глава мир жениса касымбеку. вместе тем глава государство поручить привести в порядок работа продвижение экспорта. «быть государства, который мы сейчас работаем. этот работа приве

# Задача 6

Попробуем повторить то, что было сделано в задаче 4, и посмотрим, какие результаты теперь покажут классификаторы.

In [17]:
from sklearn.feature_extraction.text import CountVectorizer 

vectorizer = CountVectorizer(min_df = 0, lowercase = True) 
vectorizer.fit(X_train_l_new)
X_train = vectorizer.transform(X_train_l_new).toarray()

test_vectorizer = CountVectorizer(min_df = 0, lowercase = True, vocabulary=vectorizer.vocabulary_) 
test_vectorizer.fit(X_test_l_new)
X_test = vectorizer.transform(X_test_l_new).toarray()

print(X_train[:10])
print(X_test[:10])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [18]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

linsvc_clf = LinearSVC(C=5, loss="hinge", penalty = 'l2', multi_class = 'ovr')

param_grid = {'C': list(range(1, 5)), 'loss': ['hinge', 'squared_hinge']}
grid_search_cv = GridSearchCV(LinearSVC(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred1 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred1)

print(accuracy_score(dataY_test, y_pred1))

0.5633333333333334


SVC не улучшил показатель точности. Наоборот, значение accuracy слегка снизилось.

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

rnd_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
rnd_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('RandomForestClassifier', RandomForestClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(RandomForestClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred2 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred2)

print(accuracy_score(dataY_test, y_pred2))
print(rnd_clf.feature_importances_)
print(rnd_clf.predict_proba(X_test))
np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['RandomForestClassifier'].feature_importances_ != 0)[0]]

0.4633333333333333
[0. 0. 0. ... 0. 0. 0.]
[[0.16329369 0.48942821 0.3472781 ]
 [0.17107837 0.48446587 0.34445577]
 [0.18208728 0.49078546 0.32712725]
 [0.16405509 0.49501811 0.34092679]
 [0.16712357 0.48274948 0.35012696]
 [0.16298725 0.49216888 0.34484387]
 [0.16524758 0.49242061 0.34233181]
 [0.17173444 0.49057179 0.33769377]
 [0.17039799 0.49511649 0.33448552]
 [0.16031051 0.48801273 0.35167675]
 [0.16125158 0.5076699  0.33107853]
 [0.1512482  0.49554885 0.35320295]
 [0.17365487 0.49670907 0.32963606]
 [0.16447747 0.49914479 0.33637774]
 [0.16524758 0.49242061 0.34233181]
 [0.20909574 0.4688521  0.32205216]
 [0.17418459 0.49549219 0.33032322]
 [0.16135264 0.49595681 0.34269055]
 [0.2707198  0.46203295 0.26724725]
 [0.16912015 0.49404501 0.33683483]
 [0.16911379 0.4913258  0.33956041]
 [0.17178251 0.4810213  0.34719619]
 [0.16453734 0.49022536 0.3452373 ]
 [0.16433864 0.48880387 0.34685749]
 [0.16818285 0.49855751 0.33325964]
 [0.163631   0.48290491 0.35346409]
 [0.1584973  0.488811

array(['02', '023', '03', ..., 'япония', 'қазақстан', 'ҳукумат'],
      dtype='<U29')

Классификатор Random Forest достиг примерно такого же уровня точности, что и без предобработки.

In [29]:
from sklearn.ensemble import ExtraTreesClassifier

ext_clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
ext_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('ExtraTreesClassifier', ExtraTreesClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(ExtraTreesClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred3 = grid_search_cv.predict(X_test)

print(accuracy_score(dataY_test, y_pred3))
print(ext_clf.feature_importances_)
print(ext_clf.predict_proba(X_test))
np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['ExtraTreesClassifier'].feature_importances_ != 0)[0]]

0.8766666666666667
[0. 0. 0. ... 0. 0. 0.]
[[0.16711212 0.83288788]
 [0.1686411  0.8313589 ]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.1686411  0.8313589 ]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.18539103 0.81460897]
 [0.17543108 0.82456892]
 [0.16535878 0.83464122]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.1736363  0.8263637 ]
 [0.1686411  0.8313589 ]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.1686901  0.8313099 ]
 [0.16711212 0.83288788]
 [0.17087393 0.82912607]
 [0.19209775 0.80790225]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.17535878 0.82464122]
 [0.19868684 0.80131316]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.17210972 0.82789028]
 [0.168

array(['000', '01', '04', ..., 'январе', 'январь', 'января'], dtype='<U32')

Самый лучший результат показал здесь классификатор Extra Trees.

Попробуем также выполнить задачу бинарной классификации.

In [21]:
y = {'negative': -1, 'neutral': 1, 'positive': 1} 
data['y'] = data['sentiment'].map(lambda x: y[x])

data_Y = data['y'].values
data_X = data['text'].values

dataY_train, dataY_test, dataX_train, dataX_test = train_test_split(data_Y, data_X, test_size=0.3)
dataY_train, dataY_test, dataX_train, dataX_test = dataY_train[:700], dataY_test[:300], dataX_train[:700], dataX_test[:300]

In [22]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
rnd_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('RandomForestClassifier', RandomForestClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(RandomForestClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred2 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred2)

print(accuracy_score(dataY_test, y_pred2))
print(rnd_clf.feature_importances_)
print(rnd_clf.predict_proba(X_test))
print(classification_report(y_true=dataY_test, y_pred=y_pred2))

np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['RandomForestClassifier'].feature_importances_ != 0)[0]]

0.8766666666666667
[0. 0. 0. ... 0. 0. 0.]
[[0.16982263 0.83017737]
 [0.17840654 0.82159346]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.1740095  0.8259905 ]
 [0.176361   0.823639  ]
 [0.16310269 0.83689731]
 [0.16159364 0.83840636]
 [0.16310269 0.83689731]
 [0.17113738 0.82886262]
 [0.16310269 0.83689731]
 [0.16849734 0.83150266]
 [0.17155645 0.82844355]
 [0.16310269 0.83689731]
 [0.19066833 0.80933167]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16533161 0.83466839]
 [0.16159364 0.83840636]
 [0.16310269 0.83689731]
 [0.17161425 0.82838575]
 [0.16646324 0.83353676]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.17693027 0.82306973]
 [0.17815876 0.82184124]
 [0.16722502 0.83277498]
 [0.17132267 0.82867733]
 [0.16310269 0.83689731]
 [0.16310269 0.83689731]
 [0.17155645 0.82844355]
 [0.17168651 0.82831349]
 [0.163

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array(['01', '04', '047', '09', '10', '10369', '121', '13', '1306',
       '1314', '1498', '156', '16', '167', '17', '19', '20', '2009',
       '2015', '2016', '2017', '21', '210', '2305', '24', '243св', '25',
       '259', '28', '288', '30', '325', '38', '41', '466', '467', '50',
       '51', '53', '548', '57427', '58', '583', '592', '61', '6928',
       '727', '728', '738', '99', 'aaa', 'asbn_dispute_case_120117',
       'b270bn', 'bus', 'camonitor', 'carplay', 'ctrl', 'development',
       'edinogo', 'enter', 'facebook', 'files', 'glavnaa', 'html', 'http',
       'inform', 'iwf', 'kapital', 'kase', 'kaspi', 'kaznex', 'kb', 'kz',
       'kселл', 'lse', 'matching', 'ne', 'web', 'xvii', 'абдижаббарова',
       'абышевой', 'аварийную', 'автодорожной', 'автозаводов',
       'автомобилей', 'автопрома', 'автопроме', 'автор', 'авторынок',
       'автосборочные', 'агромашхолдинг', 'адиль', 'административной',
       'акимом', 'актау', 'активности', 'активов', 'акций',
       'акционерного', 

Результат чуть-чуть выше, чем без предобработки.

In [23]:
from sklearn.ensemble import ExtraTreesClassifier

ext_clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
ext_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('ExtraTreesClassifier', ExtraTreesClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(ExtraTreesClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred3 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred3)

print(accuracy_score(dataY_test, y_pred3))
print(ext_clf.feature_importances_)
print(ext_clf.predict_proba(X_test))
print(classification_report(y_true=dataY_test, y_pred=y_pred3))

np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['ExtraTreesClassifier'].feature_importances_ != 0)[0]]

0.8766666666666667
[0. 0. 0. ... 0. 0. 0.]
[[0.16711212 0.83288788]
 [0.1686411  0.8313589 ]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.1686411  0.8313589 ]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.18539103 0.81460897]
 [0.17543108 0.82456892]
 [0.16535878 0.83464122]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.1736363  0.8263637 ]
 [0.1686411  0.8313589 ]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.1686901  0.8313099 ]
 [0.16711212 0.83288788]
 [0.17087393 0.82912607]
 [0.19209775 0.80790225]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.17535878 0.82464122]
 [0.19868684 0.80131316]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.16711212 0.83288788]
 [0.17210972 0.82789028]
 [0.168

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array(['000', '01', '03', ..., 'январе', 'января', 'ячеек'], dtype='<U32')

Такой же результат, как и без предобработки.

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('LogisticRegression', LogisticRegression(penalty='l1'))]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'C': list(range(1, 5)), 'penalty': ['l1', 'l2']}
grid_search_cv = GridSearchCV(LogisticRegression(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred4 = grid_search_cv.predict(X_test)

print(accuracy_score(dataY_test, y_pred4))

print(classification_report(y_true=dataY_test, y_pred=y_pred4))

np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['LogisticRegression'].coef_ != 0)[1]]

0.78
             precision    recall  f1-score   support

         -1       0.00      0.00      0.00        66
          1       0.78      1.00      0.88       234

avg / total       0.61      0.78      0.68       300



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array(['01', '1332', '2015', '2017', '21', '26', '60', '90', 'com',
       'kase', 'kz', 'today', 'vedomosti', 'автомобиль', 'агентства',
       'акишев', 'акции', 'акционеров', 'алматы', 'аналитика', 'анна',
       'ао', 'аппарат', 'астана', 'атырау', 'байтерек', 'банк', 'банка',
       'безопасности', 'билеты', 'бишимбаев', 'более', 'будут', 'бывшего',
       'был', 'была', 'были', 'было', 'вагонов', 'ведомства', 'вернуться',
       'вице', 'во', 'возможность', 'вот', 'время', 'все', 'всего',
       'выручка', 'главы', 'говорит', 'года', 'году', 'города', 'граждан',
       'данные', 'декабря', 'дел', 'детей', 'деятельности', 'директоров',
       'для', 'должен', 'дом', 'другие', 'других', 'единиц', 'ему',
       'енпф', 'ерденаева', 'же', 'за', 'завода', 'заседания',
       'заявление', 'здравоохранения', 'или', 'имеет', 'иране',
       'источник', 'итогам', 'казахстан', 'казахстана', 'казахстане',
       'казахстанских', 'казахстанской', 'казинвестбанка', 'ккм',
       'кодекса', 'к

Результат стал лучше лишь немного.

Лучшие результаты для бинарной классификации показали Random Forest и Extra Trees классификаторы.

# Задача 7

Используя классификатор, который возвращает вероятности принадлежности к классу, рассмотрим несколько сообщений, в которых классификатор больше всего не уверен (то есть степень уверенности в одним из классов колеблется от 0.5 до 0.6).

Как можно судить по работе классификаторов Extra Trees и Random Forest, очень многие сообщения получили вероятность класса neutral = 0.5. Возможно, это зависит от уровня точности модели.

Одно из таких сообщений - с индексом 10. Predict_proba классификатора Random Forest для одного из классов = 0.5076699. Также рассмотрим тексты, например, с индексами 27 (0.50432991) и 30 (0.50716492).

In [27]:
print(dataX_test[10])


Долгожданный план, названный Министерством национальной экономики «Стратегией развития Казахстана до 2025 года», наконец-то увидел свет. Сколько копий было сломано вокруг него, сколько экспертов просиживало в приемной министра долгими часами, только бы внести свои коррективы в светлое будущее.
Но будущее оказалось туманным, по крайней мере, именно такой вывод можно сделать, изучив документ. Все наполеоновские планы по типу «догнать и перегнать», видимо, Миннацэкономики оставило в прошлом и решило не сильно усложнять себе задачу, обозначив максимальным коридором ежегодного развития экономики скромные 3%. Об 11-12%, как это бывало в лучшие годы нашей жизни, уже никто и не говорит. Скромность, конечно, украшает г-на Бишимбаева, но явно не в вопросах экономического развития страны.
Итак, что получилось в сухом остатке.
В качестве ключевого индикатора Стратегии 2025 предлагается увеличение реального ВВП к 2025 году на 35%. Среднегодовые темпы роста экономики заявлены от 2,8% в 2015-2020 го

In [29]:
print(dataX_test[27])

Алматы. 6 января. КазТАГ - Ксения Бондал. Основанием для возбуждения уголовного дела по сделке Единого накопительного пенсионного фонда (ЕНПФ) стало заявление Национального банка РК, сообщил начальник управления по защите прав потребителей финансовых услуг и внешних коммуникаций Нацбанка Александр Терентьев. 

«Ранее мы эту информацию не озвучивали, за это мы получили всякие кривотолки и осуждения. Основанием для возбуждения уголовного дела стало заявление Нацбанка. Нам эта сделка (по покупке облигаций со стороны ЕНПФ - КазТАГ) показалась сомнительной, поэтому Нацбанк обратился 25 ноября 2016 года с письмом в правоохранительные органы. Это пока все, что я могу вам сообщить. Следствие идет, проверка проводится, думаю, в ближайшее время вы информацию про данному уголовному делу получите», - сказал А.Терентьев на пресс-конференции в пятницу. 

Как сообщалось, правоохранительными органами Казахстана с 21 декабря ведется проверка деятельности АО «ЕНПФ». Проверка осуществляется по операциям,

In [30]:
print(dataX_test[30])

Авиакомпания SCAT подводит статистику по своим направлениям, согласно которой Грузия заняла одно из лидирующих мест. Открытый впервые в истории взаимоотношений между Республикой Казахстан и Республикой Грузия рейс Актау — Батуми показал самый стремительный рост пассажиропотока в летнем сезоне со средней загрузкой 85 процентов. 

Исторический рейс Актау — Тбилиси, выполняемый авиакомпанией на протяжении 18 лет, уже за десять месяцев 2016 года показал 12-процентное увеличение пассажиропотока по сравнению с годовым показателем 2015 года, несмотря на открытие одного из самых популярных сезонных рейсов Актау — Батуми и других конкурирующих направлений. 

В связи с этим авиакомпания приняла решение о запуске акции в преддверии зимнего горнолыжного сезона в Грузии. Все пассажиры, приобретая билеты на любое направление авиакомпании SCAT в период с 21 ноября по 4 декабря, могут выиграть выходные на двоих в Тбилиси с предоставлением проживания и насыщенной культурной программой. Дополнительными 

Какими же могут быть причины неуверенности классификатора?

При подробном рассмотрении можно слелать вывод, что на это могут влиять такие факторы, как:
1) недостаточное количество в тексте слов, которых классификатор считает важными для определения класса
2) непосредственно длина текста, которая и сокращает вероятность попадания в него важных характеристик

3) Несовершенность модели.
Рассмотрим, например, разницу между словами, определенными как важные, логистической регрессией и классификатором Rantom forest. Многие из этих слов в двух разных моделях совпадают. Но это более общие слова, которые могли бы встретиться в текстах разной эмоциональной окрашенности. Например, Казахстан, Астана, год, назнание месяцев и валют. 

Однако, также среди общей тематики слов встречались те темы, которые уже, возможно, могли бы повлиять на определение тональности. Это слова из тематического поля "криминал":уголовный, заержан, злоупотребление, запрещено. Также слова с положительной семантикой: уют, развитие. 

В целом классификатор Rantom forest предложил более обширный список значимых слов.

# Задача 8

# Три класса

In [4]:
import numpy as np
import os
import pandas as pd

#загружаем дата-сет
data = pd.read_json('train.json', encoding = 'utf-8')
data.head(10)

from sklearn.model_selection import train_test_split

#присваиваем классам числовое значение
y = {'negative': -1, 'neutral': 0, 'positive': 1} 
data['y'] = data['sentiment'].map(lambda x: y[x])

#делим данные на x и y, в x будут тексты, а в у - класс
data_Y = data['y'].values
data_X = data['text'].values

#делим данные на тренировочную и тестовую выборку, тестовая выборка - 30% (здесь взята только часть примеров, а не весь корпус - это связано с мощностью компьютера)
dataY_train, dataY_test, dataX_train, dataX_test = train_test_split(data_Y, data_X, test_size=0.3, shuffle=True)
dataY_train, dataY_test, dataX_train, dataX_test = dataY_train[:700], dataY_test[:300], dataX_train[:700], dataX_test[:300]

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(dataX_train)
X_train = vectorizer.transform(dataX_train)
X_test = vectorizer.transform(dataX_test)

In [19]:
print(X_train[0])

  (0, 47827)	0.04592616781897563
  (0, 47394)	0.04747150142653724
  (0, 47063)	0.06579911699316304
  (0, 45974)	0.0646904895561301
  (0, 45471)	0.042086765306051044
  (0, 45003)	0.0757628274234735
  (0, 44997)	0.15556164982217482
  (0, 44994)	0.06128096838561277
  (0, 44818)	0.05060330304823705
  (0, 44498)	0.06888695875843028
  (0, 44497)	0.06037042101624428
  (0, 44002)	0.05626891198805237
  (0, 43128)	0.07078097220831826
  (0, 42561)	0.03806637658314542
  (0, 42415)	0.1235584506923133
  (0, 42281)	0.04602432873841278
  (0, 42278)	0.038651571312605326
  (0, 42050)	0.08427936516565951
  (0, 41996)	0.019875242144220567
  (0, 41794)	0.035267648551449794
  (0, 41398)	0.05185388327405827
  (0, 41389)	0.06450457691090924
  (0, 40761)	0.10012090530468541
  (0, 40146)	0.07420982018179162
  (0, 40114)	0.07929750995050427
  :	:
  (0, 6789)	0.06226443446613227
  (0, 6572)	0.09556515085654306
  (0, 6567)	0.07193988130205384
  (0, 6540)	0.026922956502342995
  (0, 6392)	0.027944939124643044
  (0, 

In [10]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

linsvc_clf = LinearSVC(C=5, loss="hinge", penalty = 'l2', multi_class = 'ovr')

#применим gridsearch для поиска оптимальных значений гиперпараметров
param_grid = {'C': list(range(1, 5)), 'loss': ['hinge', 'squared_hinge']}
grid_search_cv = GridSearchCV(LinearSVC(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred1 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred1)

print(accuracy_score(dataY_test, y_pred1))

0.6833333333333333


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer 

rnd_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
rnd_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('RandomForestClassifier', RandomForestClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(RandomForestClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred2 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred2)

print(accuracy_score(dataY_test, y_pred2))
print(rnd_clf.feature_importances_)
print(rnd_clf.predict_proba(X_test))
np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['RandomForestClassifier'].feature_importances_ != 0)[0]]

0.5633333333333334
[0. 0. 0. ... 0. 0. 0.]
[[0.16159957 0.49351333 0.3448871 ]
 [0.16829744 0.49050427 0.34119829]
 [0.15587527 0.48078298 0.36334175]
 [0.15880474 0.49031772 0.35087754]
 [0.16252933 0.4872002  0.35027048]
 [0.15674013 0.48736882 0.35589105]
 [0.15674095 0.46702465 0.3762344 ]
 [0.17666683 0.48833367 0.3349995 ]
 [0.16202447 0.47396703 0.36400849]
 [0.15371823 0.48339899 0.36288278]
 [0.17604258 0.4896253  0.33433212]
 [0.16006547 0.46553381 0.37440073]
 [0.15953028 0.48782164 0.35264808]
 [0.16277234 0.47100352 0.36622414]
 [0.15710314 0.47554664 0.36735022]
 [0.16362416 0.49035706 0.34601879]
 [0.15501456 0.47199663 0.37298881]
 [0.15319429 0.46807136 0.37873435]
 [0.27111319 0.44381088 0.28507593]
 [0.16346861 0.49277022 0.34376117]
 [0.18696586 0.47118567 0.34184847]
 [0.15973218 0.48251131 0.35775651]
 [0.15711181 0.47925272 0.36363547]
 [0.17726725 0.49455407 0.32817868]
 [0.15752609 0.4818869  0.36058701]
 [0.16871097 0.47427148 0.35701755]
 [0.17475922 0.471264

array(['000', '000м3', '003', ..., 'қатновни', 'үйі', 'әділет'],
      dtype='<U80')

In [13]:
from sklearn.ensemble import ExtraTreesClassifier

ext_clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
ext_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('ExtraTreesClassifier', ExtraTreesClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(ExtraTreesClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred3 = grid_search_cv.predict(X_test)

print(accuracy_score(dataY_test, y_pred3))
print(ext_clf.feature_importances_)
print(ext_clf.predict_proba(X_test))
np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['ExtraTreesClassifier'].feature_importances_ != 0)[0]]

0.5533333333333333
[0. 0. 0. ... 0. 0. 0.]
[[0.1627648  0.49825986 0.33897534]
 [0.16437397 0.47574178 0.35988426]
 [0.1640533  0.48326445 0.35268226]
 [0.16587978 0.48493638 0.34918384]
 [0.16576831 0.48630032 0.34793137]
 [0.16750799 0.48123964 0.35125238]
 [0.16792081 0.48439208 0.34768711]
 [0.18039934 0.4879355  0.33166516]
 [0.16321491 0.47779352 0.35899157]
 [0.16587978 0.48493638 0.34918384]
 [0.17885004 0.48041063 0.34073933]
 [0.16312603 0.47134621 0.36552777]
 [0.16761418 0.47811545 0.35427037]
 [0.16175816 0.48051353 0.35772831]
 [0.16692111 0.48892699 0.34415191]
 [0.17148223 0.48492038 0.34359739]
 [0.163888   0.46526328 0.37084872]
 [0.16484944 0.48534988 0.34980069]
 [0.2702264  0.43600455 0.29376905]
 [0.16672134 0.48583868 0.34743998]
 [0.16105867 0.48790919 0.35103215]
 [0.16799323 0.48242858 0.34957819]
 [0.16346069 0.47905555 0.35748377]
 [0.17869532 0.48048068 0.340824  ]
 [0.16768661 0.47588137 0.35643202]
 [0.16630689 0.48355721 0.3501359 ]
 [0.16587978 0.484936

array(['000', '003', '01', ..., 'қазақстан', 'қалындық', 'үкімет'],
      dtype='<U80')

# Два класса

In [14]:
y = {'negative': -1, 'neutral': 1, 'positive': 1} 
data['y'] = data['sentiment'].map(lambda x: y[x])

data_Y = data['y'].values
data_X = data['text'].values

dataY_train, dataY_test, dataX_train, dataX_test = train_test_split(data_Y, data_X, test_size=0.3)
dataY_train, dataY_test, dataX_train, dataX_test = dataY_train[:700], dataY_test[:300], dataX_train[:700], dataX_test[:300]

In [15]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
rnd_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('RandomForestClassifier', RandomForestClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(RandomForestClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred2 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred2)

print(accuracy_score(dataY_test, y_pred2))
print(rnd_clf.feature_importances_)
print(rnd_clf.predict_proba(X_test))
print(classification_report(y_true=dataY_test, y_pred=y_pred2))

np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['RandomForestClassifier'].feature_importances_ != 0)[0]]

0.78
[0. 0. 0. ... 0. 0. 0.]
[[0.14430561 0.85569439]
 [0.15261464 0.84738536]
 [0.14430561 0.85569439]
 [0.14999497 0.85000503]
 [0.14430561 0.85569439]
 [0.14637446 0.85362554]
 [0.14625963 0.85374037]
 [0.16189603 0.83810397]
 [0.14430561 0.85569439]
 [0.15022067 0.84977933]
 [0.17568061 0.82431939]
 [0.14430561 0.85569439]
 [0.15966858 0.84033142]
 [0.17436406 0.82563594]
 [0.15302318 0.84697682]
 [0.15711546 0.84288454]
 [0.15868836 0.84131164]
 [0.16006585 0.83993415]
 [0.15139726 0.84860274]
 [0.16792974 0.83207026]
 [0.15555647 0.84444353]
 [0.14430561 0.85569439]
 [0.15300503 0.84699497]
 [0.18388506 0.81611494]
 [0.14715278 0.85284722]
 [0.14717263 0.85282737]
 [0.15030426 0.84969574]
 [0.14430561 0.85569439]
 [0.14430561 0.85569439]
 [0.14717263 0.85282737]
 [0.15157992 0.84842008]
 [0.15022067 0.84977933]
 [0.14430561 0.85569439]
 [0.1735438  0.8264562 ]
 [0.14657027 0.85342973]
 [0.14430561 0.85569439]
 [0.14430561 0.85569439]
 [0.15001981 0.84998019]
 [0.14657027 0.853429

NameError: name 'classification_report' is not defined

In [16]:
from sklearn.ensemble import ExtraTreesClassifier

ext_clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=100, max_depth=2, random_state=0)
ext_clf.fit(X_train, dataY_train)

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('ExtraTreesClassifier', ExtraTreesClassifier())]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'max_depth': list(range(3, 8)), 'n_estimators': list(range(3, 8))}
grid_search_cv = GridSearchCV(ExtraTreesClassifier(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred3 = grid_search_cv.predict(X_test)
accuracy_score(dataY_test, y_pred3)

print(accuracy_score(dataY_test, y_pred3))
print(ext_clf.feature_importances_)
print(ext_clf.predict_proba(X_test))
print(classification_report(y_true=dataY_test, y_pred=y_pred3))

np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['ExtraTreesClassifier'].feature_importances_ != 0)[0]]

0.7766666666666666
[0. 0. 0. ... 0. 0. 0.]
[[0.15091678 0.84908322]
 [0.15609285 0.84390715]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15943048 0.84056952]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15445905 0.84554095]
 [0.15091678 0.84908322]
 [0.17641102 0.82358898]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.14950982 0.85049018]
 [0.15091678 0.84908322]
 [0.17308693 0.82691307]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15608852 0.84391148]
 [0.16508507 0.83491493]
 [0.15091678 0.84908322]
 [0.15940381 0.84059619]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15541389 0.84458611]
 [0.15177659 0.84822341]
 [0.15091678 0.84908322]
 [0.15608272 0.84391728]
 [0.15091678 0.84908322]
 [0.15958203 0.84041797]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15091678 0.84908322]
 [0.15445905 0.84554095]
 [0.150

NameError: name 'classification_report' is not defined

In [18]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

our_pipeline = Pipeline( [
    ('CountVectorizer', CountVectorizer()),
    ('LogisticRegression', LogisticRegression(penalty='l1'))]
)

our_pipeline.fit(dataX_train, dataY_train)

param_grid = {'C': list(range(1, 5)), 'penalty': ['l1', 'l2']}
grid_search_cv = GridSearchCV(LogisticRegression(), param_grid, cv = 3)
grid_search_cv.fit(X_train, dataY_train)

y_pred4 = grid_search_cv.predict(X_test)

print(accuracy_score(dataY_test, y_pred4))

print(classification_report(y_true=dataY_test, y_pred=y_pred4))

np.array(our_pipeline.named_steps['CountVectorizer'].get_feature_names())[np.where(our_pipeline.named_steps['LogisticRegression'].coef_ != 0)[1]]

0.78
             precision    recall  f1-score   support

         -1       0.00      0.00      0.00        66
          1       0.78      1.00      0.88       234

avg / total       0.61      0.78      0.68       300



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array(['01', '1332', '2015', '2017', '21', '26', '60', '90', 'com',
       'kase', 'kz', 'today', 'vedomosti', 'автомобиль', 'агентства',
       'акишев', 'акции', 'акционеров', 'алматы', 'анна', 'ао', 'аппарат',
       'астана', 'атырау', 'байтерек', 'банк', 'банка', 'безопасности',
       'билеты', 'бишимбаев', 'более', 'будут', 'бывшего', 'был', 'была',
       'были', 'было', 'вагонов', 'ведомства', 'вернуться', 'вице', 'во',
       'возможность', 'вот', 'время', 'все', 'всего', 'выручка', 'главы',
       'говорит', 'года', 'году', 'города', 'граждан', 'данные',
       'декабря', 'дел', 'детей', 'деятельности', 'директоров', 'для',
       'должен', 'дом', 'другие', 'других', 'единиц', 'ему', 'енпф',
       'ерденаева', 'же', 'за', 'завода', 'заседания', 'заявление',
       'здравоохранения', 'или', 'имеет', 'иране', 'источник', 'итогам',
       'казахстан', 'казахстана', 'казахстане', 'казахстанских',
       'казахстанской', 'казинвестбанка', 'ккм', 'кодекса', 'конструкции',
       